In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
pip install "pinecone-client[grpc]"

In [3]:
import os
from pinecone import Pinecone, ServerlessSpec
load_dotenv()
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

c:\Users\navya\anaconda\envs\rag\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '1abe6d43e6baa36b429c00f5eae880b2', 'Date': 'Tue, 27 Aug 2024 13:24:15 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [4]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Weller',
  'subject': 'Introduction to Psychology',
  'stars': 4,
  'review': "Dr. Weller is a fantastic professor. Her lectures are engaging and she really cares about her students' learning."},
 {'professor': 'Professor Michael Nguyen',
  'subject': 'Calculus I',
  'stars': 3,
  'review': "Professor Nguyen's class is challenging, but he is very helpful during office hours. The grading can be a bit harsh, however."},
 {'professor': 'Dr. Sophia Ramirez',
  'subject': 'Organic Chemistry',
  'stars': 5,
  'review': 'Dr. Ramirez is an amazing professor. She explains complex topics clearly and is always available to help students. I highly recommend taking her class.'},
 {'professor': 'Professor David Goldstein',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': "Professor Goldstein's class is well-organized and he is very knowledgeable about the subject matter. The assignments can be time-consuming, but they are valuable learning experien

In [17]:
pip install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'


In [19]:
pip activate conda

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "activate"



In [5]:
from sentence_transformers import SentenceTransformer
import torch

processed_data = []
model = SentenceTransformer('all-MiniLM-L6-v2')

for review in data['reviews']:
    embedding = model.encode(review['review'], convert_to_tensor=True)
    
    # Pad or truncate the embedding to 8192 dimensions
    if len(embedding) < 384:
        embedding = torch.nn.functional.pad(embedding, (0, 384 - len(embedding)), mode='constant', value=0)
    elif len(embedding) > 384:
        embedding = embedding[:384]
    
    processed_data.append({
        "values": embedding.tolist(),
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\navya\anaconda\envs\rag\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\navya\anaconda\envs\rag\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\navya\anaconda\envs\rag\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\navya\anaconda\envs\rag\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Us

In [6]:
processed_data[0]

{'values': [-0.005701903253793716,
  -0.02479683980345726,
  0.03689616918563843,
  -0.006063878536224365,
  -0.06112971529364586,
  0.031153948977589607,
  -0.07165687531232834,
  0.08030837029218674,
  -0.01627936400473118,
  -0.005643927492201328,
  -0.017285017296671867,
  0.12241912633180618,
  -0.08644145727157593,
  0.03330835700035095,
  -0.04486880823969841,
  0.025657664984464645,
  -0.023832956328988075,
  -0.018659492954611778,
  -0.016236687079072,
  -0.06707756966352463,
  -0.012002374976873398,
  0.05417356267571449,
  0.06287179887294769,
  0.03160524740815163,
  -0.030581098049879074,
  -0.0014491124311462045,
  0.05421441048383713,
  -0.0913119688630104,
  0.03238588199019432,
  -0.02278653159737587,
  -0.04637722671031952,
  0.03648103401064873,
  -0.08369536697864532,
  -0.020050961524248123,
  -0.07678963243961334,
  0.019580483436584473,
  0.010050063021481037,
  0.10293146967887878,
  0.027486024424433708,
  0.04228077828884125,
  -0.032387834042310715,
  -0.0161

In [7]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [8]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}